In [1]:
import torch
import os
import numpy as np

In [2]:
one_train_dir = os.path.join('dataset', 'part_one_dataset', 'train_data')
one_eval_dir = os.path.join('dataset', 'part_one_dataset', 'eval_data')
two_train_dir = os.path.join('dataset', 'part_two_dataset', 'train_data')
two_eval_dir = os.path.join('dataset', 'part_two_dataset', 'eval_data')

In [3]:
# domains = [{} for _ in range(20)]
# eval_domains = [{} for _ in range(20)]

# for i in range(10):
#     domains[i] = torch.load(os.path.join(one_train_dir, f'{i+1}_train_data.tar.pth'), weights_only=False)
#     domains[i+10] = torch.load(os.path.join(two_train_dir, f'{i+1}_train_data.tar.pth'), weights_only=False)
#     eval_domains[i] = torch.load(os.path.join(one_eval_dir, f'{i+1}_eval_data.tar.pth'), weights_only=False)
#     eval_domains[i+10] = torch.load(os.path.join(two_eval_dir, f'{i+1}_eval_data.tar.pth'), weights_only=False)
    
#     domains[i]['data'] = domains[i]['data'].reshape(2500, -1)
#     domains[i+10]['data'] = domains[i+10]['data'].reshape(2500, -1)
#     eval_domains[i]['data'] = eval_domains[i]['data'].reshape(2500, -1)
#     eval_domains[i+10]['data'] = eval_domains[i+10]['data'].reshape(2500, -1)

In [4]:
train_paths = [os.path.join(one_train_dir, f'{i+1}_train_data.tar.pth') for i in range(10)] + [os.path.join(two_train_dir, f'{i+1}_train_data.tar.pth') for i in range(10)]
eval_paths = [os.path.join(one_eval_dir, f'{i+1}_eval_data.tar.pth') for i in range(10)] + [os.path.join(two_eval_dir, f'{i+1}_eval_data.tar.pth') for i in range(10)]

eval_domains = [torch.load(eval_paths[i], weights_only=False) for i in range(20)]

## Model

In [5]:
from lwp import LWP

## Training and Predictions

In [6]:
from sklearn.metrics import accuracy_score
import pandas as pd

model = LWP(distance_metric='minkowski', p = 3)

df = pd.DataFrame()

for idx,train_path in enumerate(train_paths):
    
    domain = torch.load(train_path, weights_only=False)    
    x_test = domain['data'].reshape(2500, -1)
    y_pred = model.predict(x_test) if 'targets' not in domain else domain['targets']
    
    model.fit(x_test, y_pred)
    print(model.class_counts)
    del domain
    
    scores = []
    for eval_domain in eval_domains[:idx+1]:
        
        features = eval_domain['data'].reshape(2500, -1)
        labels = eval_domain['targets']
        
        preds = model.predict(features)
        acc = accuracy_score(labels, preds)
        
        scores.append(acc)
        
    
    df[f'Domain {idx+1}'] = scores + [np.nan] * (20 - len(scores))

{0: 253, 1: 243, 2: 255, 3: 244, 4: 262, 5: 236, 6: 250, 7: 253, 8: 254, 9: 250}
{0: 732, 1: 401, 2: 400, 3: 319, 4: 413, 5: 554, 6: 789, 7: 374, 8: 460, 9: 558}
{0: 1149, 1: 614, 2: 625, 3: 443, 4: 609, 5: 840, 6: 1230, 7: 510, 8: 694, 9: 786}
{0: 1503, 1: 868, 2: 844, 3: 589, 4: 819, 5: 1127, 6: 1603, 7: 674, 8: 926, 9: 1047}
{0: 1838, 1: 1142, 2: 1079, 3: 729, 4: 1031, 5: 1362, 6: 2015, 7: 860, 8: 1166, 9: 1278}
{0: 2179, 1: 1442, 2: 1358, 3: 881, 4: 1241, 5: 1613, 6: 2373, 7: 1048, 8: 1393, 9: 1472}
{0: 2495, 1: 1740, 2: 1645, 3: 1042, 4: 1451, 5: 1863, 6: 2705, 7: 1236, 8: 1643, 9: 1680}
{0: 2797, 1: 2031, 2: 1932, 3: 1237, 4: 1641, 5: 2138, 6: 3022, 7: 1435, 8: 1884, 9: 1883}
{0: 3084, 1: 2335, 2: 2221, 3: 1429, 4: 1851, 5: 2378, 6: 3338, 7: 1619, 8: 2138, 9: 2107}
{0: 3351, 1: 2649, 2: 2553, 3: 1593, 4: 2076, 5: 2624, 6: 3649, 7: 1807, 8: 2391, 9: 2307}
{0: 3643, 1: 2931, 2: 2856, 3: 1777, 4: 2263, 5: 2841, 6: 3983, 7: 2004, 8: 2669, 9: 2533}
{0: 3930, 1: 3208, 2: 3158, 3: 1961,

## Evaluation

In [7]:
df

,Domain 1,Domain 2,Domain 3,Domain 4,Domain 5,Domain 6,Domain 7,Domain 8,Domain 9,Domain 10,Domain 11,Domain 12,Domain 13,Domain 14,Domain 15,Domain 16,Domain 17,Domain 18,Domain 19,Domain 20
0,0.2636,0.2588,0.2528,0.2564,0.2520,0.2488,0.2476,0.2476,0.2492,0.2484,0.2464,0.2452,0.2428,0.2428,0.2412,0.2400,0.2392,0.2356,0.2348,0.2344
1,NaN,0.2512,0.2460,0.2444,0.2424,0.2388,0.2372,0.2360,0.2360,0.2344,0.2324,0.2296,0.2288,0.2284,0.2268,0.2264,0.2260,0.2244,0.2244,0.2224
2,NaN,NaN,0.2340,0.2348,0.2340,0.2288,0.2268,0.2268,0.2264,0.2248,0.2228,0.2208,0.2224,0.2216,0.2200,0.2196,0.2176,0.2160,0.2156,0.2172
3,NaN,NaN,NaN,0.2404,0.2416,0.2420,0.2396,0.2368,0.2344,0.2332,0.2328,0.2332,0.2332,0.2328,0.2320,0.2316,0.2308,0.2288,0.2280,0.2272
4,NaN,NaN,NaN,NaN,0.2736,0.2716,0.2696,0.2676,0.2640,0.2608,0.2584,0.2580,0.2560,0.2552,0.2556,0.2544,0.2552,0.2536,0.2524,0.2492
5,NaN,NaN,NaN,NaN,NaN,0.2248,0.2228,0.2200,0.2200,0.2184,0.2204,0.2196,0.2180,0.2180,0.2164,0.2172,0.2156,0.2168,0.2156,0.2128
6,NaN,NaN,NaN,NaN,NaN,NaN,0.2392,0.2356,0.2344,0.2352,0.2336,0.2320,0.2324,0.2312,0.2308,0.2300,0.2292,0.2268,0.2260,0.2228
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2460,0.2448,0.2448,0.2432,0.2440,0.2420,0.2416,0.2412,0.2396,0.2412,0.2396,0.2396,0.2404
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2492,0.2476,0.2436,0.2412,0.2416,0.2412,0.2420,0.2400,0.2408,0.2408,0.2404,0.2400
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2340,0.2328,0.2328,0.2288,0.2256,0.2252,0.2264,0.2232,0.2228,0.2232,0.2244
